In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
file_path = "../data/20250727nigata6R"

with open(file_path+".html", "r", encoding="utf-8") as file:
    html = file.read()

In [22]:
soup = BeautifulSoup(html, "html.parser")

In [23]:
# 出馬表テーブルを抽出
table = soup.select_one("#Speed_List table")

In [24]:
# テーブルのヘッダを抽出
header_rows = table.find_all("thead")[0].find_all("tr")
headers = []

# ヘッダーの1行目と2行目からカラム名を取得（結合）
for th in header_rows[0].find_all("th"):
    rowspan = th.get("rowspan")
    colspan = th.get("colspan")
    text = th.get_text(strip=True).replace("\n", "")
    if rowspan == "2":
        headers.append(text)
    elif colspan:  # 3列分の「近走成績」など
        span_headers = [span_th.get_text(strip=True) for span_th in header_rows[1].find_all("th")]
        headers.extend(span_headers)


In [25]:
# テーブルのヘッダを抽出
header_rows = table.find_all("thead")[0].find_all("tr")
headers = []

In [26]:
# ヘッダーの1行目と2行目からカラム名を取得（結合）
for th in header_rows[0].find_all("th"):
    rowspan = th.get("rowspan")
    colspan = th.get("colspan")
    text = th.get_text(strip=True).replace("\n", "")
    if rowspan == "2":
        headers.append(text)
    elif colspan:  # 3列分の「近走成績」など
        span_headers = [span_th.get_text(strip=True) for span_th in header_rows[1].find_all("th")]
        headers.extend(span_headers)

In [27]:
# データ行を抽出
rows = []
for tr in table.select("tbody tr"):
    cols = tr.find_all("td")
    row_data = []
    for td in cols:
        # セレクトボックスなどは除き、テキストのみ取得
        span = td.find("span", class_="Sort_Function_Data_Hidden")
        if span:
            try:
                time_index = span.get_text(strip=True)[1:]
                time_index = float(time_index)
                row_data.append(time_index)
            except ValueError:
                umamei = span.get_text(strip=True)
                row_data.append(umamei)
            
        else:
            row_data.append(td.get_text(strip=True).replace("\n", ""))
    rows.append(row_data)

In [31]:
# pandasでデータフレームに変換
df = pd.DataFrame(rows, columns=headers)
df = df.drop(columns=["印", "単勝オッズ","人気"])

In [32]:
df

,枠,馬番,馬名,性齢,斤量,騎手,最高,５走平均,距離,コ｜ス,3走,2走,前走
0,1,1,ヘツレクリーニ,牡4,55.0,内田博幸,84.0,79.0,84.0,67.0,79.0,78.0,84.0
1,2,2,テルハイス,牡6,52.0,嶋田純次,57.0,55.0,0.0,0.0,69.0,57.0,53.0
2,3,3,アンヒハレント,牡5,54.0,石川裕紀人,96.0,90.0,84.0,63.0,96.0,94.0,84.0
3,4,4,テンクマール,牡3,55.0,Ｃ．ルメール,102.0,91.0,102.0,0.0,0.0,79.0,102.0
4,5,5,コートアリシアン,牝3,54.0,菅原明良,101.0,97.0,101.0,90.0,98.0,101.0,100.0
5,6,6,メタルスヒート,牡5,56.0,三浦皇成,97.0,91.0,98.0,79.0,96.0,86.0,89.0
6,6,7,テラメリタ,牝4,54.0,戸崎圭太,101.0,91.0,101.0,0.0,84.0,86.0,98.0
7,7,8,シヤンハンマーク,牡4,55.0,原優介,92.0,84.0,92.0,0.0,90.0,92.0,73.0
8,7,9,エイムフオーエース,牡4,55.0,Ｒ．キング,98.0,86.0,95.0,0.0,90.0,85.0,75.0
9,8,10,メイシヨウヨソラ,牝4,54.0,吉田豊,100.0,94.0,100.0,0.0,102.0,100.0,88.0


In [33]:
df.to_csv(file_path + ".csv", index=False, encoding="utf-8")

In [ ]:
<span class="Sort_Function_Data_Hidden">991</span>